In [1]:
import cv2
import numpy as np
import math
import random
import time

current_milli_time = lambda: int(round(time.time() * 1000))

def get_rect(w=100,h=50):
    point_one = (random.randint(0,400-w),random.randint(0,500-h))
    point_two = (point_one[0] + w,point_one[1]+h)
    return point_one, point_two

def new_line(line):
    global output,x1,x2,robot_point,catch_point,goal,line_speed,robot_speed,cnt_start,angle
    goal = False
    output = line.copy()
    robot_point = (500,500)
    x1,x2 = get_rect()
    cnt = get_center(x1,x2)
    cnt_start = cnt
    catch_point = get_catch_point(cnt[0],500,cnt[1],line_speed,robot_speed)
    angle = - 180 - (math.atan2(robot_point[1] - catch_point[1], robot_point[0] - catch_point[0]) * 180/math.pi)

    
def clear_detect(last_rect):
    global x1,x2,output
    output = line.copy()
    x1,x2 = last_rect
    
def refresh_line(x,y,robot,angle):
    global x1,x2,output,center,robot_point,catch_point,cnt_start
    output = line.copy()
    
    x1,x2 = x,y
    
    catch_point =  get_catch_point(cnt_start[0],500,cnt_start[1],line_speed,robot_speed)
    angle = angle
    robot_point = robot
    center = ((x[0]+y[0])//2,(x[1]+y[1])//2)

#получение координаты точки встречи коробки и робота
def get_catch_point(x0,l0,y0,u,v):

    y0 = l0 - y0


    a=(l0-x0)*v
    b=v*y0
    c=u*y0
    sqr=math.sqrt(a**2+b**2)
    fi=math.asin(a/sqr)
    alfa=math.acos(c/sqr)+fi
    
#     x = y0/(math.tan(alfa))+l0
    
    t=y0/(v*math.sin(alfa))
    xobj=x0+u*t
    yobj=y0
    xrob=(l0+v*math.cos(alfa)*t)
    yrob=v*math.sin(alfa)*t
    y=(xrob-l0)*math.tan(alfa)
#     print(f"xobj {int(xobj)} yobj {int(yobj)} xrob {int(xrob)} yrob {int(yrob)} Y(XRob) {y}")
#     if xrob<400:
#         xrob = 400
    print(f"Координаты точки: {xrob},{abs(y0-l0)}")
    return (int(xrob),int(abs(y0-l0)))
  
        
def get_center(x,y):
    return ((x[0]+y[0])//2,(x[1]+y[1])//2)
def clear():
    global output
    output = line.copy()
    
    
def distf(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))
    

In [4]:
fourcc = cv2.VideoWriter_fourcc(*'MP4V') 
v_out = None


line = np.zeros((500,1200,3))
line[:,398:400] = (255,255,255) 

# положение робота
robot_point = (500,500)

angle = 0

#скорости конвейера и робота
line_speed = 300
robot_speed = 300

#координаты левого верхнего и правого нижнего угла коробки для отрисовки
x1,x2 = get_rect()

#центр коробки x,y
center = get_center(x1,x2)
cnt_start = center

#ожидаемая координата встречи робота и коробки
catch_point = get_catch_point(x0=center[0],l0=500,y0 = center[1],u=line_speed,v=robot_speed)

cv2.namedWindow('image')
cv2.createTrackbar('Line Speed', 'image', line_speed, 500,lambda x : x)


output = line.copy()

last_time = current_milli_time()

detected = False
goal = False
robot_done = False


while True:
    line_speed = cv2.getTrackbarPos('Line Speed', 'image')
    key = cv2.waitKey(1)
    last_rect = x1,x2
    last_robot_point = robot_point
    
    
    cv2.putText(output, "g - new box; d - detection", (600,480), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)
    cv2.putText(output, f"robot-dot {round(distf((600,500),catch_point),2)} box-dot {round(distf(center,catch_point),2)}", (600,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)
    

    cv2.circle(output, (int(robot_point[0]),int(robot_point[1])), 10,(0,0,255),15)
    cv2.rectangle(output, x1, x2, (0, 255, 255), 3)
    
    if detected:
       
        center = get_center(x1,x2)

        angle =  - 180 - (math.atan2(robot_point[1] - catch_point[1], robot_point[0] - catch_point[0]) * 180/math.pi)
        cv2.circle(output, center, 10,(0,255,0),3)
        cv2.circle(output, catch_point, 10,(0,255,0),3)
        
        
    
        if (current_milli_time()-last_time)>=10:
            
            
           
            if not goal:
                    obj_speed = line_speed/100
                    if obj_speed<0:
                        obj_speed=1
                    rob_speed = robot_speed/100
                    if rob_speed<0:
                        rob_speed=1
               
                    x1 = x1[0]+int(obj_speed),x1[1]
                    x2 = x2[0]+int(obj_speed),x2[1]
                   
                    robot_point = int(robot_point[0])+ int(rob_speed) * math.cos(angle *math.pi/180),int(robot_point[1])- int(rob_speed) * math.sin(angle*math.pi/180)
                    
                   
            if distf(center,catch_point)<line_speed//100:
                goal = True
                
                
            
            if (current_milli_time()-last_time)>=100:
                print(angle)
                refresh_line(x1,x2,robot_point,angle)
                    
            last_time = current_milli_time()
            
           
            
    if v_out is None:
        v_out = cv2.VideoWriter('test {}.mp4'.format(time.time()/1000),fourcc, 60.0, (1200,500),3)
    v_out.write(np.uint8(output))
    cv2.imshow("image",output)
    
    
    if key == ord('g'):
        new_line(line)
        detected = False
    if key == ord('d'):
        if detected:
            clear_detect(last_rect)
            detected = False
        else: 
            detected = True
        
    if key == ord('q'):
#         v_out.release()
        break 
        
cv2.destroyAllWindows()

Координаты точки: 370.3486394557822,399


In [5]:
 v_out.release()